<a href="https://colab.research.google.com/github/maddieiyengar/cs315/blob/main/HeapHHApportionment_inclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title key_object
#!/usr/bin/env python3
# key_object.py

# Introduction to Algorithms, Fourth edition
# Linda Xiao

#########################################################################
#                                                                       #
# Copyright 2022 Massachusetts Institute of Technology                  #
#                                                                       #
# Permission is hereby granted, free of charge, to any person obtaining #
# a copy of this software and associated documentation files (the       #
# "Software"), to deal in the Software without restriction, including   #
# without limitation the rights to use, copy, modify, merge, publish,   #
# distribute, sublicense, and/or sell copies of the Software, and to    #
# permit persons to whom the Software is furnished to do so, subject to #
# the following conditions:                                             #
#                                                                       #
# The above copyright notice and this permission notice shall be        #
# included in all copies or substantial portions of the Software.       #
#                                                                       #
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,       #
# EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF    #
# MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND                 #
# NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS   #
# BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN    #
# ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN     #
# CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE      #
# SOFTWARE.                                                             #
#                                                                       #
#########################################################################

class KeyObject:
	"""Used for testing anything that requires a key."""

	def __init__(self, string, key):
		self.string = string
		self.key = key

	@staticmethod
	def get_key(x):
		return x.key

	@staticmethod
	def set_key(x, key):
		x.key = key

	def __gt__(self, obj2):
		return self.key > obj2.key

	def __str__(self):
		return self.string

In [7]:
# @title Heap code
#!/usr/bin/env python3
# max_heap.py

# Introduction to Algorithms, Fourth edition
# Linda Xiao and Tom Cormen

#########################################################################
#                                                                       #
# Copyright 2022 Massachusetts Institute of Technology                  #
#                                                                       #
# Permission is hereby granted, free of charge, to any person obtaining #
# a copy of this software and associated documentation files (the       #
# "Software"), to deal in the Software without restriction, including   #
# without limitation the rights to use, copy, modify, merge, publish,   #
# distribute, sublicense, and/or sell copies of the Software, and to    #
# permit persons to whom the Software is furnished to do so, subject to #
# the following conditions:                                             #
#                                                                       #
# The above copyright notice and this permission notice shall be        #
# included in all copies or substantial portions of the Software.       #
#                                                                       #
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,       #
# EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF    #
# MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND                 #
# NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS   #
# BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN    #
# ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN     #
# CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE      #
# SOFTWARE.                                                             #
#                                                                       #
#########################################################################

"""Base class for MaxHeap and MinHeap."""


class Heap:
    def __init__(self, compare, array, get_key_func=None, dict=None):
        """Initialize a heap with an array and heap size.

        Arguments:
        compare -- comparison function: greater-than for a max-heap, less-than for a min-heap
        array -- array of heap elements.
        get_key_func -- an optional function that returns the key for the
        objects stored. If given, may be a static function in the object class. If
        omitted, then the identity function is used.
        dict -- an optional dictionary mapping objects in the max-heap to indices.
        """
        self.compare = compare
        self.array = array
        # heap_size is the number of elements in the heap that are stored
        # in the array, defaults to all elements in array.
        self.heap_size = len(array)
        if get_key_func is None:
            self.get_key = lambda x: x
        else:
            self.get_key = get_key_func

        # If there is a dictionary mapping objects to indices, initialize it.
        # It should be empty to start.
        self.dict = dict
        if self.dict is not None:
            if len(self.dict) > 0:
                raise RuntimeError("Dictionary argument to constructor must be None or an empty dictionary.")
            for i in range(self.heap_size):
                dict[self.array[i]] = i

    def get_heap_size(self):
        """Return the size of this heap."""
        return self.heap_size

    def is_full(self):
        """Return True if this heap is full, False if not full."""
        return self.heap_size >= len(self.array)

    def get_array(self):
        """Return the array implementation of this heap."""
        return self.array

    def set_heap_size(self, size):
        """Set heap size to given size."""
        self.heap_size = size

    def parent(self, i):
        """Return the index of the parent node of i."""
        return (i-1) // 2

    def left(self, i):
        """Return the index of the left child of i."""
        return 2*i + 1

    def right(self, i):
        """Return the index of the right child of i. """
        return 2*i + 2

    def swap(self, i, j):
        """Swap two elements in an array."""
        if self.dict is not None:
            self.dict[self.array[i]] = j
            self.dict[self.array[j]] = i
        self.array[i], self.array[j] = self.array[j], self.array[i]

    def heapify(self, i):
        """Maintain the heap property.

        Argument:
        i -- index of the element in the heap.
        """
        l = self.left(i)
        r = self.right(i)

        if l < self.heap_size and self.compare(self.get_key(self.array[l]), self.get_key(self.array[i])):
            swap_with = l
        else:
            swap_with = i

        if r < self.heap_size and self.compare(self.get_key(self.array[r]), self.get_key(self.array[swap_with])):
            swap_with = r

        if swap_with != i:
            self.swap(i, swap_with)
            self.heapify(swap_with)

    def build_heap(self):
        """Convert a list or numpy array into a heap."""
        # Run heapify on all roots of the tree, from ((heap_size // 2) - 1) to 0.
        self.heap_size = len(self.array)
        for i in range((len(self.array) // 2) - 1, -1, -1):
            self.heapify(i)

    def __str__(self):
        """Return the heap as an array."""
        return ", ".join(str(x) for x in self.array[:self.heap_size])

    def is_heap(self):
        """Verify that the array or list represents a heap."""
        # From root node to last internal node.
        for i in range(0, self.heap_size // 2):
            # Check the left child.
            if self.compare(self.get_key(self.array[self.left(i)]), self.get_key(self.array[i])):
                return False
            # If there is a right child, check it.
            if self.right(i) < self.heap_size and \
                    self.compare(self.get_key(self.array[self.right(i)]), self.get_key(self.array[i])):
                return False

        return True

In [4]:
# @title Heap Priority Queue
#!/usr/bin/env python3
# min_heap_priority_queue.py

# Introduction to Algorithms, Fourth edition
# Linda Xiao and Tom Cormen

#########################################################################
#                                                                       #
# Copyright 2022 Massachusetts Institute of Technology                  #
#                                                                       #
# Permission is hereby granted, free of charge, to any person obtaining #
# a copy of this software and associated documentation files (the       #
# "Software"), to deal in the Software without restriction, including   #
# without limitation the rights to use, copy, modify, merge, publish,   #
# distribute, sublicense, and/or sell copies of the Software, and to    #
# permit persons to whom the Software is furnished to do so, subject to #
# the following conditions:                                             #
#                                                                       #
# The above copyright notice and this permission notice shall be        #
# included in all copies or substantial portions of the Software.       #
#                                                                       #
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,       #
# EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF    #
# MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND                 #
# NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS   #
# BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN    #
# ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN     #
# CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE      #
# SOFTWARE.                                                             #
#                                                                       #
#########################################################################

"""Base class for MaxHeapPriorityQueue and MinHeapPriorityQueue."""

class HeapPriorityQueue:

    def __init__(self, compare, temp_insert_value, get_key_func, set_key_func=None):
        """Initialize minimum priority queue implemented with a heap.

        Arguments:
        compare -- comparison function: greater-than for a max-heap priority queue,
        less-than for a min-heap priority queue
        temp_insert_value -- temporary value given to objects upon insertion, then
        changed to the actual value of the object
        get_key_func -- required function that returns the key for the
        objects stored. May be a static function in the object class.
        set_key_func -- optional function that sets the key for the objects
        stored. May be a static function in the object class.
        """

        # Dictionary to map array objects to array indices.
        # Mapping might not take worst-case time O(1).
        self.dict = {}

        # self.get_key function used to get key of object.
        self.get_key = get_key_func

        # self.set_key function used to set key of object.
        self.set_key = set_key_func

        # Initialize to empty heap.
        self.heap = Heap(compare, [], self.get_key, self.dict)
        self.compare = compare
        self.temp_insert_value = temp_insert_value

    def get_heap(self):
        """Return heap, used in testing."""
        return self.heap

    def get_size(self):
        """Return the number of objects in the priority queue."""
        return self.heap.get_heap_size()

    def top_of_heap(self):
        """Return the object at the top of the heap."""
        if self.heap.get_heap_size() <= 0:  # error if heap is empty
            raise RuntimeError("Heap underflow.")
        return self.heap.get_array()[0]

    def extract_top(self):
        """Return and delete the top element in a heap."""
        top = self.top_of_heap()

        # Move the last object in heap to the root position.
        last_obj = self.heap.get_array()[self.heap.get_heap_size()-1]
        self.heap.get_array()[0] = last_obj
        self.dict[last_obj] = 0

        # Remove the old top object.
        del self.dict[top]
        self.heap.set_heap_size(self.heap.get_heap_size() - 1)

        # Restore the heap property.
        self.heap.heapify(0)

        # Return the top item, which was extracted.
        return top

    def update_key(self, x, k):
        """Update the key of object x to value k.
        Assumption: The caller has already verified that the new value is OK.

        Arguments:
        x -- object whose key has been changed
        k -- new key of x
        """
        if self.set_key is not None:
            self.set_key(x, k)

        # Get the index from the dictionary.
        i = self.dict[x]

        # Compare the value with parents up the heap to place in the correct position.
        while i > 0 and \
                self.compare(self.get_key(self.heap.get_array()[i]),
                             self.get_key(self.heap.get_array()[self.heap.parent(i)])):
            # Exchange positions and continue if the element should head toward the root.
            self.heap.swap(i, self.heap.parent(i))
            i = self.heap.parent(i)

    def insert(self, x):
        """Insert x into the heap.  Grows the heap as necessary.

        Arguments:
        x -- object to insert
        """

        # Increment the heap size.
        self.heap.set_heap_size(self.heap.get_heap_size() + 1)

        k = self.get_key(x)

        if self.set_key is not None:
            self.set_key(x, self.temp_insert_value)

        # Insert x into the array and the dictionary.
        self.heap.get_array().insert(self.heap.get_heap_size() - 1, x)
        self.dict[x] = self.heap.get_heap_size() - 1

        # Maintain the heap property.
        self.update_key(x, k)

    def is_heap(self):
        """Verify that the array or list represents a heap."""
        return self.heap.is_heap()

    def __str__(self):
        """Return the heap as an array."""
        return str(self.heap)

In [5]:
# @title Max Heap Priority Queue
#!/usr/bin/env python3
# max_heap_priority_queue.py

# Introduction to Algorithms, Fourth edition
# Linda Xiao and Tom Cormen

#########################################################################
#                                                                       #
# Copyright 2022 Massachusetts Institute of Technology                  #
#                                                                       #
# Permission is hereby granted, free of charge, to any person obtaining #
# a copy of this software and associated documentation files (the       #
# "Software"), to deal in the Software without restriction, including   #
# without limitation the rights to use, copy, modify, merge, publish,   #
# distribute, sublicense, and/or sell copies of the Software, and to    #
# permit persons to whom the Software is furnished to do so, subject to #
# the following conditions:                                             #
#                                                                       #
# The above copyright notice and this permission notice shall be        #
# included in all copies or substantial portions of the Software.       #
#                                                                       #
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,       #
# EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF    #
# MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND                 #
# NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS   #
# BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN    #
# ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN     #
# CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE      #
# SOFTWARE.                                                             #
#                                                                       #
#########################################################################

class MaxHeapPriorityQueue(HeapPriorityQueue):

	def __init__(self, get_key_func, set_key_func=None):
		"""Initialize a maximum priority queue implemented with a heap.

		Arguments:
		get_key_func -- required function that returns the key for the
		objects stored. May be a static function in the object class.
		set_key_func -- optional function that sets the key for the objects
		stored. May be a static function in the object class.
		"""
		HeapPriorityQueue.__init__(self, lambda x, y: x > y, float('-inf'), get_key_func, set_key_func)

	def maximum(self):
		"""Return the object with the maximum key in a heap."""
		return self.top_of_heap()

	def extract_max(self):
		"""Return and delete the object with the maximum value in a heap."""
		return self.extract_top()

	def increase_key(self, x, k):
		"""Increase the key of object x to value k.  Error if k is less than x's current key.
			Update the heap structure appropriately.

		Arguments:
		x -- object whose key has been increased
		k -- new key of x
		"""

		if k < self.get_key(x):
			raise RuntimeError("Error in increase_key: new key " + str(k)
							   + " is less than current key " + str(x.get_key()))

		# Make the changes in the heap.
		self.update_key(x, k)

	def insert(self, x):
		"""Insert x into the max heap.  Grows the heap as necessary."""
		HeapPriorityQueue.insert(self, x)

In [12]:
# @title Huntington-Hill Apportionment Implementation

import math
if __name__ == "__main__":
  import numpy as np

  # Enter the total number of Representatives
  R = 435

  # A list with the population of each state, given in the same order
  # (alphabetically is the most natural) as the list with the state names.
  statePop = [5024279,733391,7151502,3011524,39538223,5773714,3605944,989948,21538187,
              10711908,1455271,1839106,12812508,6785528,3190369,2937880,4505836,4657757,
              1362359,6177224,7029917,10077331,5706494,2961279,6154913,1084225,1961504,
              3104614,1377529,9288994,2117522,20201249,10439388,779094,11799448,3959353,
              4237256,13002700,1097379,5118425,886667,6910840,29145505,3271616,643077,
              8631393,7705281,1793716,5893718,576851]
  stateAbbrev = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado","Connecticut",
                 "Delaware","Florida","Georgia","Hawaii","Idaho","Illinois","Indiana","Iowa",
                 "Kansas","Kentucky","Louisiana","Maine","Maryland","Massachusetts","Michigan",
                 "Minnesota","Mississippi","Missouri","Montana","Nebraska","Nevada","NewHampshire",
                 "NewJersey","NewMexico","NewYork","NorthCarolina","NorthDakota","Ohio","Oklahoma",
                 "Oregon","Pennsylvania","RhodeIsland","SouthCarolina","SouthDakota","Tennessee",
                 "Texas","Utah","Vermont","Virginia","Washington","WestVirginia","Wisconsin","Wyoming"]

  actual_apportionment = [7,1,9,4,52,8,5,1,28,14,2,2,17,9,4,4,6,6,2,8,9,13,8,4,8,2,3,4,2,12,3,26,
                        14,1,15,5,6,17,2,7,1,9,38,4,1,11,10,2,8,1]
  # Set up the list containing the number of representatives for each state
  # in the same state order as above.
  n = len(statePop)
  stateRep = [1]*n

  # Define the Max-Priority Queue.
  pq1 = MaxHeapPriorityQueue(KeyObject.get_key, KeyObject.set_key)

  # Fill the Max-Priority Queue with the state name as the element
  # and the associated geometric mean as the key value.
  # Initially, each state has 1 representative apportioned, so the
  # initial geometric mean will be the population/( sqrt(1+2) ).
  for i in range(n):
    pq1.insert(KeyObject( stateAbbrev[i],  statePop[i]/math.sqrt(2)  ))

  # The Huntington-Hill algorithm for apportionment.
  # Loop for as long as there are representatives left to apportion.
  for i in range(R-n):
    # Extract the state at the top of the priority queue.
    s =pq1.extract_max()

    # Convert the string of the state name to the location of the state in
    # the list with the number of representatives.
    stateLoc = stateAbbrev.index(s.string)

    # Increment this state's number of representatives.
    stateRep[ stateLoc ] += 1

    # Recalculate the new key value, which uses the geometric mean.
    priority = statePop[stateLoc]/math.sqrt(stateRep[stateLoc]*(stateRep[stateLoc]+1))

    # Insert this state back into the Max-Priority Queue with its updated key.
    pq1.insert( KeyObject( stateAbbrev[stateLoc] , priority) )

  # Once done with the loop, output the list of representatives.
  print(stateRep)

  # Verify results
print("\nVerification:")
print(f"{'State':<20} {'Calculated':>12} {'Actual':>10} {'Match':>8}")
print("-"*60)

all_match = True
for i in range(n):
    match = "Yes" if stateRep[i] == actual_apportionment[i] else "No"
    if stateRep[i] != actual_apportionment[i]:
        all_match = False
    print(f"{stateAbbrev[i]:<20} {stateRep[i]:>12} {actual_apportionment[i]:>10} {match:>8}")

if all_match:
    print("\nSUCCESS: All values match actual 2020 apportionment!")
else:
    print("\nDISCREPANCY: Some values differ from actual 2020 apportionment.")

[7, 1, 9, 4, 52, 8, 5, 1, 28, 14, 2, 2, 17, 9, 4, 4, 6, 6, 2, 8, 9, 13, 8, 4, 8, 2, 3, 4, 2, 12, 3, 26, 14, 1, 15, 5, 6, 17, 2, 7, 1, 9, 38, 4, 1, 11, 10, 2, 8, 1]

Verification:
State                  Calculated     Actual    Match
------------------------------------------------------------
Alabama                         7          7      Yes
Alaska                          1          1      Yes
Arizona                         9          9      Yes
Arkansas                        4          4      Yes
California                     52         52      Yes
Colorado                        8          8      Yes
Connecticut                     5          5      Yes
Delaware                        1          1      Yes
Florida                        28         28      Yes
Georgia                        14         14      Yes
Hawaii                          2          2      Yes
Idaho                           2          2      Yes
Illinois                       17         17      Yes
Indi